(30%) Documentation of the problem formulation: Clearly describe the objective function, the variables, the constraints, and the assumptions involved in formulating the problem.

Objective function: 
In this project, Drag force and noice are taken into consideration. Then we have the following problem instead:




(40%) Programming: Like for homeworks, please push you code to your github repo. Please comment your code so that it is useful to you in the future.

(30%) Analysis of the results: Please document the convergence and the optimal solutions (e.g., the state trajectory if it is a control problem).

(Bonus 20%) Formulation of a problem different from rocket landing: The PyTorch framework can also be used for other engineering problems, e.g., for structure design with nonlinear mechanical properties. You get 20 bonus points for solving your own problems that are at least at the same level of difficulty as rocket landing.
In this project, we add three different aspects to make the problem become more reality:



In [1]:
import logging
import math
import random
import numpy as np
import time
import torch as t
import torch.nn as nn
from torch import optim
from torch.nn import utils
import matplotlib.pyplot as plt

logger = logging.getLogger(__name__)

In [2]:
# environment parameters

FRAME_TIME = 0.1  # time interval
GRAVITY_ACCEL = 0.12  # gravity constant
BOOST_ACCEL = 0.18  # thrust constant
DRAG_ACCEL = 0.0001  # drag accel constant 
#noise normal  1 for d; 2 for v
mean1=0
mean2=0
variance1=0
variance2=0

#rocket_mass=10
# # the following parameters are not being used in the sample code
# PLATFORM_WIDTH = 0.25  # landing platform width
# PLATFORM_HEIGHT = 0.06  # landing platform height
# ROTATION_ACCEL = 20  # rotation constant

In [3]:
class Dynamics(nn.Module):

    def __init__(self):
        super(Dynamics, self).__init__()

    @staticmethod
    def forward(state, action):

        """
        action: thrust or no thrust
        state[0] = y
        state[1] = y_dot
        """
        
        # Apply gravity
        # Note: Here gravity is used to change velocity which is the second element of the state vector
        # Normally, we would do x[1] = x[1] + gravity * delta_time
        # but this is not allowed in PyTorch since it overwrites one variable (x[1]) that is part of the computational graph to be differentiated.
        # Therefore, I define a tensor dx = [0., gravity * delta_time], and do x = x + dx. This is allowed... 
        delta_state_gravity = t.tensor([0., GRAVITY_ACCEL * FRAME_TIME], dtype=t.float32)
        v = t.matmul(state,t.tensor([0,1], dtype=t.float32))
        # Thrust  & drag
        # Note: Same reason as above. Need a 2-by-1 tensor.
        delta_state1 = BOOST_ACCEL * FRAME_TIME * t.tensor([0., -1.]) * action    #thrust 
#         adrag = DRAG_ACCEL_COE * v * v 
#         drag=adrag*rocket_mass
       # if state[1] < 0
        #delta_state2 = DRAG_ACCEL_COE * v * v * FRAME_TIME * t.tensor([0., -1.]) * action  #add drag accele into v
        delta_state2 = DRAG_ACCEL_COE * v * v * FRAME_TIME * t.tensor([0., -1.]) * action
        # Update velocity
        state = state + delta_state1 + delta_state2  + delta_state_gravity   # 3 accelerations matter
        
        # Noise
        w = t.tensor([1, 0]) * t.tensor(np.random.normal(mean1, variance1, 1))
        u = t.tensor([0, 1]) * t.tensor(np.random.normal(mean2, variance2, 1)) 
        noise = w + u
        # Update state
        # Note: Same as above. Use operators on matrices/tensors as much as possible. Do not use element-wise operators as they are considered inplace.
        step_mat = t.tensor([[1., 0],
                            [FRAME_TIME, 1.]])
        state = t.matmul(state, step_mat) + noise
        

        return state.float()

In [4]:
####need change 

class Controller(nn.Module):

    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to you
        """
        super(Controller, self).__init__()
        self.network = nn.Sequential(
            #nn.Linear(dim_input, dim_hidden, dtype=t.float32),
            nn.Linear(dim_input, dim_hidden, dtype=t.float32),
            nn.Linear(dim_hidden, dim_output, dtype=t.float32))
            # You can add more layers here 

    def forward(self, state):
        action = self.network(state)
       # print(action)
        return action

In [5]:
class Simulation(nn.Module):

    def __init__(self, controller, dynamics, T):
        super(Simulation, self).__init__()
        self.state = self.initialize_state()
        self.controller = controller
        self.dynamics = dynamics
        self.T = T
        self.action_trajectory = []
        self.state_trajectory = []
        
    def forward(self, state):
        self.action_trajectory = []
        self.state_trajectory = []
        for _ in range(T):
            action = self.controller(state)
            state = self.dynamics(state, action)
            self.action_trajectory.append(action)
            self.state_trajectory.append(state)
        return self.error(state)

    @staticmethod
    def initialize_state():
        state = [4.,0.]  # TODO: need batch of initial states
        return t.tensor(state, requires_grad=False).float()
    


#         state = np.ones((30, 2))
#         for i in range(state.shape[0]):
#             state[i][0] = np.random.uniform(1, 2.5, 1)
#             state[i][1] = 0
#         return t.tensor(state, requires_grad=False).float()
    
    def error(self, state):
        #return t.mean(state ** 2)
        return state[0]**2 + state[1]**2

In [6]:
# set up the optimizer
# Note:
# 0. LBFGS is a good choice if you don't have a large batch size (i.e., a lot of initial states to consider simultaneously)
# 1. You can also try SGD and other momentum-based methods implemented in PyTorch
# 2. You will need to customize "visualize"
# 3. loss.backward is where the gradient is calculated (d_loss/d_variables)
# 4. self.optimizer.step(closure) is where gradient descent is done

class Optimize:
    def __init__(self, simulation):
        self.simulation = simulation
        self.parameters = simulation.controller.parameters()
        self.optimizer = optim.SGD(self.parameters, lr=0.01, momentum=1)

    def step(self):
        def closure():
            loss = self.simulation(self.simulation.state)
            self.optimizer.zero_grad()
            loss.backward()
            return loss
        self.optimizer.step(closure)
        return closure()
    
    def train(self, epochs):
        for epoch in range(epochs):
            loss = self.step()
            print('[%d] loss: %.3f' % (epoch + 1, loss))
            self.visualize()
    def visualize(self):
        data = np.array([self.simulation.state_trajectory[i].detach().numpy() for i in range(self.simulation.T)])
        x = data[:, 0]
        y = data[:, 1]
        plt.plot(x, y)
        plt.show()
#     def visualize(self):
#         data = np.array([[self.simulation.state_trajectory[i][j].detach().numpy() for i in range(self.simulation.T)] for j in range(30)])
#         for i in range(30):
#             x = data[i, :, 0]
#             y = data[i, :, 1]
#             plt.plot(x, y)
#         plt.title('Position and Velocity Changeable for Rocket Landing')
#         plt.xlabel('Rocket Position(m)')
#         plt.ylabel('Rocket Velocity(m/s)')
#         plt.show()

In [7]:
# Now it's time to run the code!

T = 100  # number of time steps
dim_input = 2  # state space dimensions
dim_hidden = 6  # latent dimensions
dim_output = 1  # action space dimensions


d = Dynamics()  # define dynamics
c = Controller(dim_input, dim_hidden, dim_output)  # define controller
s = Simulation(c, d, T)  # define simulation
o = Optimize(s)  # define optimizer
o.train(40)  # solve the optimization problem

AttributeError: module 'torch.nn' has no attribute 'tanh'